In [7]:
import glob
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from scipy.stats import pearsonr
import statistics
from sklearn.model_selection import train_test_split,GroupShuffleSplit
from sklearn.ensemble import (
    RandomForestRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor)
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import (
    cross_val_score,
    RepeatedKFold,
    RandomizedSearchCV,
    KFold,
    train_test_split,
    GridSearchCV,
    GroupKFold)
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.svm import SVR
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
from math import sqrt
import warnings
from scipy import stats
from sklearn.exceptions import ConvergenceWarning

# Suppress ConstantInputWarning
warnings.filterwarnings("ignore", category=stats.ConstantInputWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [8]:
class base:
    def __init__(self, **kwargs):
        self.features = kwargs["features"]
        self.response = kwargs["response"]

    def preProcess_features(self, **kwargs):
        if kwargs["rescale_type"] == "norm":
            standardized = preprocessing.StandardScaler()
            features_processed = standardized.fit_transform(np.array(self.features))
        elif kwargs["rescale_type"] == "minmax":
            norm = preprocessing.MinMaxScaler()
            features_processed = norm.fit_transform(np.array(self.features))
        else:
            print(str(kwargs["rescale_type"])+ "rescaling technique not implemented \n Defaulting to standardized variables")
            standardized = preprocessing.StandardScaler()
            features_processed = standardized.fit_transform(np.array(self.features))

        self.features_processed = features_processed
        return features_processed

    def predict(self, **kwargs):
        features_to_predict = kwargs["features"]
        self.predictions = self.model.predict(features_to_predict)

    def run_CVs(self, **kwargs):

        standardized = preprocessing.StandardScaler()
        norm = preprocessing.MinMaxScaler()

        x_data = kwargs["features"]
        rescale_type = kwargs['rescale_type']

        # Normalise or standardize, two different forms of rescaling
        if kwargs["rescale_type"] == "norm":
            x_data = standardized.fit_transform(np.array(x_data))
        elif kwargs["rescale_type"] == "minmax":
            x_data = norm.fit_transform(np.array(x_data))
        else:
            print("rescaling technique not implemented \n Defaulting to standardized variables")
            x_data = standardized.fit_transform(np.array(x_data))

        y_data = np.array(kwargs["response"])
        n_folds = kwargs["n_folds"]
        title = kwargs["title"]

        kf = KFold(n_splits=n_folds)
        df = {}
        fold_indices = {}

        count = 1
        for train_index, test_index in kf.split(x_data):
            X_train, X_test = x_data[train_index], x_data[test_index]
            y_train, y_test = y_data[train_index], y_data[test_index]

            model_cv = kwargs["model_type"]
            model_cv.fit(X_train, y_train)

            fold = model_cv.predict(X_test)
            df[f"Pred{count}"] = fold
            df[f"Obs{count}"] = y_test

            fold_indices[f"Train{count}"] = train_index
            fold_indices[f"Test{count}"] = test_index

            count += 1

        fig = plt.figure(figsize=(20, 15))

        count = 1

        comb_cv_obs = []
        comb_cv_preds = []

        # Wether or not to visualise the cross validation
        try:
            visualize = kwargs["visualize"]
        except:
            visualize = None

        if visualize:
            for i in range(n_folds):
                print(i)
                print(f"23{count}")
                ax = fig.add_subplot(int(n_folds/2),2,count)
                sns.regplot(x=df[f"Obs{count}"], y=df[f"Pred{count}"])
                ax.spines["right"].set_visible(False)
                ax.spines["top"].set_visible(False)
                ax.set_ylabel("Predicted")
                ax.set_xlabel("Observed")
                ax.set_title(f"Fold{count}")
                r_val, pval = spearmanr(df[f"Obs{count}"], df[f"Pred{count}"])
                r2_val = round(r_val ** 2, 2)
                x_cord, y_cord = max(df[f"Obs{count}"]) * 0.15, max(df[f"Pred{count}"])
                ax.annotate(f"$R^2 = {r2_val}$", (x_cord, y_cord))

                comb_cv_preds.extend(df[f"Pred{count}"])
                comb_cv_obs.extend(df[f"Obs{count}"])

                count += 1

            R, pVal = spearmanr(comb_cv_obs, comb_cv_preds)
            R2 = round(R ** 2, 2)

            fig.suptitle(f"{title} (Combined data $R^2$ = {R2})")
            plt.tight_layout()
            direc = os.getcwd()
            out_direc = f"{direc}"
            os.makedirs(out_direc, exist_ok=True)
            # plt.savefig(f'{out_direc}/Fold_{title}.png')
            # plt.savefig(f'{out_direc}/Fold_{title}.svg')
            # plt.show()

            fig2 = plt.figure(figsize=(12, 10))
            ax2 = fig2.add_subplot(111)
            sns.regplot(x=comb_cv_obs, y=comb_cv_preds, ax=ax2)
            ax2.spines["top"].set_visible(False)
            ax2.spines["right"].set_visible(False)
            ax2.set_title(title, pad=10)
            r, pval = spearmanr(comb_cv_obs, comb_cv_preds)
            r2 = round(r ** 2, 2)
            x_coord = max(comb_cv_obs) * 0.75
            y_coord = max(comb_cv_preds) * 0.99
            ax2.text(x_coord, y_coord, f"$R^2 = {round(r2,2)}$")
            ax2.set_xlabel("Observations", labelpad=20)
            ax2.set_ylabel("Predictions", labelpad=20)
            fig2.tight_layout()
            fig2.savefig(f'{out_direc}/{title}.png') 
            # fig2.savefig(f'{out_direc}/{title}.svg')

        return df, fold_indices

In [9]:
class LR(base):

    """ Linear Regression Model """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))

    def train_lr(self, **kwargs):
        model = LinearRegression()

        model.fit(self.features_processed, self.response)
        self.model = model
        return model

    def run_CVs(self, **kwargs):
        df, fold_indices = super().run_CVs(
            model_type=LinearRegression(),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])

        return df, fold_indices


class RF(base):

    """" Random Forest Regression Model """
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))


    def grid_search(self, **kwargs):
        n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=10)]
        max_features = [1, 2, 3, 4,5,6,7,8,9,10]
        max_depth = [int(x) for x in np.linspace(10, stop=100, num=11)]
        max_depth.append(None)
        min_samples_split = [2, 5, 10]
        min_samples_leaf = [10, 15, 20]
        bootstrap = [True, False]
        random_grid = {
            "n_estimators": n_estimators,
            "max_features": max_features,
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "min_samples_leaf": min_samples_leaf,
            "bootstrap": bootstrap}
       
        rf = RandomForestRegressor()
       
        rf_random = RandomizedSearchCV(
            estimator=rf,
            param_distributions=random_grid,
            n_iter=100,
            cv=3,
            verbose=0,
            random_state=42,
            n_jobs=-1)
        rf_random.fit(self.features_processed, self.response)
        # print(rf_random.best_params_)

        self.ran_params = rf_random.best_params_

    def train_rf(self, **kwargs):
        model = RandomForestRegressor(
            n_estimators=self.ran_params["n_estimators"],
            min_samples_leaf=self.ran_params["min_samples_leaf"],
            min_samples_split=self.ran_params["min_samples_split"],
            max_features=self.ran_params["max_features"],
            max_depth=self.ran_params["max_depth"],
            bootstrap=self.ran_params["bootstrap"])

        model.fit(self.features_processed, self.response)
        self.model = model
        
        return model

    def run_CVs(self, **kwargs):
        df, fold_indices = super().run_CVs(
            model_type=RandomForestRegressor(
                n_estimators=self.ran_params["n_estimators"],
                min_samples_leaf=self.ran_params["min_samples_leaf"],
                min_samples_split=self.ran_params["min_samples_split"],
                max_features=self.ran_params["max_features"],
                max_depth=self.ran_params["max_depth"],
                bootstrap=self.ran_params["bootstrap"]),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])

        return df, fold_indices

    def feature_importance(self, **kwargs):

        importances = self.model.feature_importances_
        self.feature_importance_std = np.std([tree.feature_importances_ for tree in self.model.estimators_], axis=0)
        return importances
    

    
    
class GB(base):

    """ Gradient Boosting Regression Model """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))

    def grid_search(self, **kwargs):

        n_estimators = [int(x) for x in np.linspace(start=100, stop=1000, num=10)]
        max_features = [1,2,3,4,5,6,7,8,9,10]
        max_depth = [int(x) for x in np.linspace(10, stop=100, num=11)]
        max_depth.append(None)
        min_samples_split = [2, 5, 10]
        min_samples_leaf = [10, 15, 20]
        bootstrap = [True, False]  
        random_grid = {
            "n_estimators": n_estimators,
            "max_features": max_features,
            "max_depth": max_depth,
            "min_samples_split": min_samples_split,
            "min_samples_leaf": min_samples_leaf}

        gb = GradientBoostingRegressor()
        
        gb_random = RandomizedSearchCV(
            estimator=gb,
            param_distributions=random_grid,
            n_iter=100,
            cv=3,
            verbose=0,
            random_state=42,
            n_jobs=-1)  
        gb_random.fit(self.features_processed, self.response)

        self.ran_params = gb_random.best_params_

    def train_gb(self, **kwargs):
        # self.grid_search()
        model = GradientBoostingRegressor(
            n_estimators=self.ran_params["n_estimators"],
            min_samples_leaf=self.ran_params["min_samples_leaf"],
            min_samples_split=self.ran_params["min_samples_split"],
            max_features=self.ran_params["max_features"],
            max_depth=self.ran_params["max_depth"])

        model.fit(self.features_processed, self.response)
        self.model = model
        
        return model

    def run_CVs(self, **kwargs):
        df, fold_indices = super().run_CVs(
            model_type=GradientBoostingRegressor(
                n_estimators=self.ran_params["n_estimators"],
                min_samples_leaf=self.ran_params["min_samples_leaf"],
                min_samples_split=self.ran_params["min_samples_split"],
                max_features=self.ran_params["max_features"],
                max_depth=self.ran_params["max_depth"]),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])

        return df, fold_indices


class PLSR(base):

    """ Partial Least Squares Regression Model """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))

    def optimum_ncomps(self, **kwargs):
        self.features = np.array(self.features)
        self.response = np.array(self.response)
        X_train, X_test, y_train, y_test = train_test_split(
            self.features,
            self.response,
            test_size=kwargs["test_size"],
            random_state=8675309)
        # Compute the validation error for each n_comp
        trait_plot = []
        for n_comp in range(1, self.features.shape[1]):
            # print(len(self.features))
            my_plsr = PLSRegression(n_components=n_comp, scale=True)
            my_plsr.fit(X_train, y_train)
            preds = my_plsr.predict(X_test)
            trait_rmse = sqrt(mean_squared_error(y_test, preds))
            trait_plot.append(trait_rmse)

        min_trait_index = trait_plot.index(min(trait_plot))

        self.min_rmse = min(trait_plot)
        self.ncomps_min_rsme = min_trait_index + 1

    def train_plsr(self):
        model = PLSRegression(n_components=self.ncomps_min_rsme, scale=True)
        model.fit(self.features_processed, self.response)
        self.model = model
        
        return model

    def run_CVs(self, **kwargs):
        df, fold_indices = super().run_CVs(
            model_type=PLSRegression(n_components=self.ncomps_min_rsme),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])
        return df, fold_indices


class SVM(base):

    """ Support Vector Regression """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))

    def train_svm(self):
        model = SVR(epsilon=2)
        model.fit(self.features_processed, self.response)
        self.model = model
        
        return model

    def run_CVs(self, **kwargs):
        df, fold_indices = super().run_CVs(
            model_type=SVR(epsilon=2),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])
        return df, fold_indices

    # Lasso
    
class Ridgemodel(base):
    
    """ lasso regression """
    
    def __init__(self, features, response, **kwargs):
        super().__init__(features=features, response=response, **kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))
    
    def paramSearch(self, **kwargs):
        self.features = np.array(self.features)
        self.response = np.array(self.response)
        
        X_train, X_test, y_train, y_test = train_test_split(
            self.features,
            self.response,
            test_size=kwargs["test_size"],
            random_state=8675309)
        
        ridge=Ridge()
        
        alphas = np.logspace(-3, 2, 100) 
        param_grid = {'alpha':alphas}
        
        grid_search = GridSearchCV(ridge, param_grid, scoring='neg_mean_squared_error', cv=5)
        grid_search.fit(X_train, y_train)
        
        self.best_alpha = grid_search.best_params_['alpha']
        
    def train_ridge(self, **kwargs):
        model=Ridge(alpha=self.best_alpha)
        model.fit(self.features_processed, self.response)
        self.model=model
        
        return model
    
    # def predict(self, **)
        
        
    def run_CVs(self,**kwargs):
        df, fold_indices = super().run_CVs(
            model_type=Ridge(alpha=self.best_alpha),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])
        return df, fold_indices

class Lassomodel(base):
    
    """ lasso regression """
    
    def __init__(self, features, response, **kwargs):
        super().__init__(features=features, response=response, **kwargs)
        self.features_processed = self.preProcess_features(rescale_type=kwargs.get("rescale_type"))
    
    def paramSearch(self, **kwargs):
        self.features = np.array(self.features)
        self.response = np.array(self.response)
        
        X_train, X_test, y_train, y_test = train_test_split(
            self.features,
            self.response,
            test_size=kwargs["test_size"],
            random_state=8675309)
        
        ridge=Lasso()
        
        alphas = np.logspace(-3, 4, 1000)
        # print(alphas)
        param_grid = {'alpha':alphas}
        
        grid_search = GridSearchCV(ridge, param_grid, scoring='neg_mean_squared_error', cv=5)
        grid_search.fit(X_train, y_train)
        
        self.best_alpha = grid_search.best_params_['alpha']
        
    def train_lasso(self, **kwargs):
        model=Lasso(alpha=self.best_alpha,max_iter=1000000)
        model.fit(self.features_processed, self.response)
        self.model=model
        
        return model
    
    def run_CVs(self,**kwargs):
        df, fold_indices = super().run_CVs(
            model_type=Lasso(alpha=self.best_alpha,max_iter=100000),
            features=kwargs["features"],
            response=kwargs["response"],
            n_folds=kwargs["n_folds"],
            title=kwargs["title"],
            visualize=kwargs["visualize"],
            rescale_type=kwargs["rescale_type"])
        return df, fold_indices

In [11]:
path='/home/schnablelab/Documents/NNSatelliteImages/Data/'
mainfolder=os.listdir(path)
Correlation=[]
for mainfolders in mainfolder:
    if not mainfolders.endswith('.csv'):
        mainfolderspath=os.path.join(path,mainfolders)
        location=os.path.basename(mainfolderspath)
        print(location)
        if location=='Lincoln' or location=='Scottsbluff':
            continue
        
        subfolder=os.listdir(mainfolderspath)
        for subfolders in subfolder:
            
            if subfolders=='Satelliteimages':
                # continue
                
                satfolderpath=os.path.join(mainfolderspath,subfolders)
                
                satsubfolders=os.listdir(satfolderpath)
                bands='Satellite Image'
                
                for satsubfolder in satsubfolders:
                    
                    if satsubfolder=='sixband':
                        
                    
                        finalsatfolderpath=os.path.join(satfolderpath,satsubfolder)
                    
                        satfiles=os.listdir(finalsatfolderpath)
                        
                        for file in satfiles:
                            if file.endswith('_genotype.csv'):
                                print(file)
                                
                                timepoint=file.split('_')[-2]
                                
                                print(timepoint)
                                
                                datafilepath=os.path.join(finalsatfolderpath,file)
                                
                                datadf=pd.read_csv(datafilepath,index_col=0)
                                # print(datadf.columns)
                                datadf=datadf.iloc[:,list(range(0, 12))+list(range(15,39))+[46]+[-1]]
                                
                                
                                datadf=datadf.dropna(subset=['totalStandCount','genotype'])
                                datadf=datadf[datadf['totalStandCount']>25]
#                                 print(datadf.shape)
#                                 continue
                                
                                # train_inds, test_inds = next(GroupShuffleSplit(test_size=.2, n_splits=5, random_state = 7).split(datadf, groups=datadf['genotype']))
                                # print(train_inds)
                                group_kfold = GroupKFold(n_splits=5)

                                # Getting the indices for splitting
                                splits = group_kfold.split(X=datadf, groups=datadf['genotype'])
                                
                                # for train, test in splits:
                                #     print(train,test)

                                
#                                 unique_genotypes = datadf['genotype'].unique()

#                                 # Shuffle the unique genotypes
#                                 np.random.shuffle(unique_genotypes)

#                                 # Partition the genotypes into five folds
#                                 fold_indices = []
#                                 fold_size = len(unique_genotypes) // 5
#                                 for i in range(5):
#                                     if i < 4:
#                                         test_genotypes = unique_genotypes[i * fold_size: (i + 1) * fold_size]
#                                     else:
#                                         test_genotypes = unique_genotypes[i * fold_size:]

#                                     test_index = datadf['genotype'].isin(test_genotypes)
#                                     train_index = ~test_index

#                                     fold_indices.append((np.where(train_index)[0], np.where(test_index)[0]))
#                                                                 # train = datadf.iloc[train_inds]
                                rep=1
                                for trainindex, testindex in splits:
                                    
                                    train=datadf.iloc[trainindex]
                                    test=datadf.iloc[testindex]
                                    
                                    features=train[train.columns[train.columns.str.contains('mean|sum|median')]]
                                    response=train['totalStandCount']
                                    
                                    testfeatures=test[test.columns[train.columns.str.contains('mean|sum|median')]]
                                    testresponse=test['totalStandCount']
                                    Preprocessing=preprocessing.StandardScaler()
                                    testfeatures=Preprocessing.fit_transform(testfeatures)
                                    
                                    ##########RandomForest###################
                                    
                                    model=RF(response=response,features=features,rescale_type="norm")
                                    model.grid_search()
                                    RFmodel=model.train_rf(response=response, features=features)
                                    
                                    results=RFmodel.predict(testfeatures)
                                    results = results.flatten()
                                    r,p=pearsonr(results,testresponse)
                                    model='RF'
                                    r_squared=r*r
                                    print(model,f'Fold:{rep}')
                                    print(r_squared)
                                    
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared, 'rep':rep})

                                    #############PLSR##############
                                    plsrmodel=PLSR(response=response, features=features,rescale_type="norm")
                                    plsrmodel.optimum_ncomps(test_size=0.1)
                                    plsrmodel1=plsrmodel.train_plsr()

                                    # Preprocessing=preprocessing.StandardScaler()
                                    # testfeatures=Preprocessing.fit_transform(testfeatures)
                                    results=plsrmodel1.predict(testfeatures)
                                    results = results.flatten()
                                    # print(len(results), len(testresponse))

                                    r,p=pearsonr(results,testresponse)
                                    model='PSLR'
                                    r_squared=r*r
                                    print(model,f'Fold:{rep}')

                                    print(r*r)
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})
                        
                                    ############################LR######################
                                    lrmodel=LR(response=response, features=features,rescale_type="norm")

                                    lrmodel1=lrmodel.train_lr(response=response, features=features)

                                    results=lrmodel1.predict(testfeatures)
                                    results=results.flatten()

                                    r,p=pearsonr(results,testresponse)
                                    r_squared=r*r
                                    model='LR'
                                    print(model,f'Fold:{rep}')
                                    print(r_squared)
                                    
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                                    ################Ridge###################

    #                                 ridgemodel=Ridgemodel(response=response, features=features,rescale_type="norm")
    #                                 ridgemodel.paramSearch(test_size=0.1)
    #                                 ridgemodel1=ridgemodel.train_ridge(response=response, features=features)

    #                                 results=ridgemodel1.predict(testfeatures)
    #                                 results=results.flatten()

    #                                 r,p=pearsonr(results,testresponse)
    #                                 print(r*r)

                                    ##################Lasso##################
                                    lassomodel=Lassomodel(response=response, features=features,rescale_type="norm")
                                    lassomodel.paramSearch(test_size=0.1)
                                    lassomodel1=lassomodel.train_lasso()

                                    results=lassomodel1.predict(testfeatures)
                                    results=results.flatten()

                                    r,p=pearsonr(results,testresponse)
                                    r_squared=r*r
                                    model='LASSO'
                                    print(model,f'Fold:{rep}')
                                    print(r*r)
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                                    ############SVM############
                                    svmmodel=SVM(response=response, features=features,rescale_type="norm")
                                    svmmodel1=svmmodel.train_svm()

                                    results=svmmodel1.predict(testfeatures)
                                    results=results.flatten()

                                    r,p=pearsonr(results,testresponse)
                                    r_squared=r*r
                                    model='SVM'
                                    print(model,f'Fold:{rep}')
                                    print(r*r)
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                                    ######GB#################

                                    gbmodel=GB(response=response, features=features,rescale_type="norm")
                                    gbmodel.grid_search()
                                    gbmodel1=gbmodel.train_gb(response=response, features=features)

                                    results=gbmodel1.predict(testfeatures)
                                    results=results.flatten()

                                    r,p=pearsonr(results,testresponse)
                                    r_squared=r*r
                                    model='GB'
                                    print(model,f'Fold:{rep}')
                                    print(r*r)
                                    Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})
                                    
                                    rep=rep+1
                                
            if subfolders=='UAV':
                # continue
                
                uavfolderpath=os.path.join(mainfolderspath,subfolders)
                
                uavfiles=os.listdir(uavfolderpath)
                bands='UAV'

                for file in uavfiles:
                    if file.endswith('_genotype.csv'):
                        print(file)

                        timepoint=file.split('_')[-2]

                        print(timepoint)

                        datafilepath=os.path.join(uavfolderpath,file)

                        uavdf=pd.read_csv(datafilepath,index_col=0)

                        uavdf=uavdf.iloc[:,list(range(0, 15))+[25]+[-1]]
                        uavdf=uavdf.dropna(subset=['totalStandCount','genotype'])
                        uavdf=uavdf[uavdf['totalStandCount']>25]
                        # print(uavdf.shape)
                        # continue
                        
                    
                        group_kfold = GroupKFold(n_splits=5)

                                # Getting the indices for splitting
                        splits = group_kfold.split(X=uavdf, groups=uavdf['genotype'])
                                
                                # for train, test in splits:
                                #     print(train,test)

                                
#                                 unique_genotypes = datadf['genotype'].unique()

#                                 # Shuffle the unique genotypes
#                                 np.random.shuffle(unique_genotypes)

#                                 # Partition the genotypes into five folds
#                                 fold_indices = []
#                                 fold_size = len(unique_genotypes) // 5
#                                 for i in range(5):
#                                     if i < 4:
#                                         test_genotypes = unique_genotypes[i * fold_size: (i + 1) * fold_size]
#                                     else:
#                                         test_genotypes = unique_genotypes[i * fold_size:]

#                                     test_index = datadf['genotype'].isin(test_genotypes)
#                                     train_index = ~test_index

#                                     fold_indices.append((np.where(train_index)[0], np.where(test_index)[0]))
#                                                                 # train = datadf.iloc[train_inds]
                        rep=1
                        # print(fold_indices)
                        print(uavdf.shape)
                        for trainindex, testindex in splits:
                            # print(trainindex)
                            max_value = trainindex.max()
                            max_testvalue=testindex.max()
                            
#                             if location=='Crawfordsville' and max_value==486:
#                                 trainindex = trainindex[trainindex != max_value]
                                
#                             if location=='Crawfordsville' and max_testvalue==486:
#                                 testindex=testindex[testindex != max_testvalue]
                            
                            train=uavdf.iloc[trainindex]
                            
                            test=uavdf.iloc[testindex]


#                             train = uavdf.iloc[train_inds]

#                             test = uavdf.iloc[test_inds]

                            features=train[train.columns[train.columns.str.contains('mean|sum|median')]]
                            response=train['totalStandCount']


                            testfeatures=test[test.columns[train.columns.str.contains('mean|sum|median')]]
                            testresponse=test['totalStandCount']
                            Preprocessing=preprocessing.StandardScaler()
                            testfeatures=Preprocessing.fit_transform(testfeatures)


                            ##########RandomForest###################

                            model=RF(response=response,features=features,rescale_type="norm")
                            model.grid_search()
                            RFmodel=model.train_rf(response=response, features=features)

                            results=RFmodel.predict(testfeatures)
                            results = results.flatten()
                            r,p=pearsonr(results,testresponse)
                            model='RF'
                            r_squared=r*r
                            print(model,f'Fold:{rep}')
                            print(r_squared)

                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                            #############PLSR##############
                            plsrmodel=PLSR(response=response, features=features,rescale_type="norm")
                            plsrmodel.optimum_ncomps(test_size=0.1)
                            plsrmodel1=plsrmodel.train_plsr()

                            # Preprocessing=preprocessing.StandardScaler()
                            # testfeatures=Preprocessing.fit_transform(testfeatures)
                            results=plsrmodel1.predict(testfeatures)
                            results = results.flatten()
                            # print(len(results), len(testresponse))

                            r,p=pearsonr(results,testresponse)
                            r_squared=r*r
                            model='PLSR'
                            print(model,f'Fold:{rep}')
                            print(r*r)
                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                            ############################LR######################
                            lrmodel=LR(response=response, features=features,rescale_type="norm")

                            lrmodel1=lrmodel.train_lr(response=response, features=features)

                            results=lrmodel1.predict(testfeatures)
                            results=results.flatten()

                            r,p=pearsonr(results,testresponse)
                            r_squared=r*r
                            model='LR'
                            print(model,f'Fold:{rep}')
                            print(r*r)
                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                            ################Ridge###################

    #                         ridgemodel=Ridgemodel(response=response, features=features,rescale_type="norm")
    #                         ridgemodel.paramSearch(test_size=0.1)
    #                         ridgemodel1=ridgemodel.train_ridge(response=response, features=features)

    #                         results=ridgemodel1.predict(testfeatures)
    #                         results=results.flatten()

    #                         r,p=pearsonr(results,testresponse)
    #                         print('ridge')
    #                         print(r*r)

                            ##################Lasso##################
                            lassomodel=Lassomodel(response=response, features=features,rescale_type="norm")
                            lassomodel.paramSearch(test_size=0.1)
                            lassomodel1=lassomodel.train_lasso()

                            results=lassomodel1.predict(testfeatures)
                            results=results.flatten()

                            r,p=pearsonr(results,testresponse)
                            r_squared=r*r
                            model='LASSO'
                            print(model,f'Fold:{rep}')
                            print(r*r)
                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                            ############SVM############
                            svmmodel=SVM(response=response, features=features,rescale_type="norm")
                            svmmodel1=svmmodel.train_svm()

                            results=svmmodel1.predict(testfeatures)
                            results=results.flatten()

                            r,p=pearsonr(results,testresponse)
                            model='SVM'
                            r_squared=r*r
                            print(model,f'Fold:{rep}')
                            print(r*r)
                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})

                            ######GB#################

                            gbmodel=GB(response=response, features=features,rescale_type="norm")
                            gbmodel.grid_search()
                            gbmodel1=gbmodel.train_gb(response=response, features=features)

                            results=gbmodel1.predict(testfeatures)
                            results=results.flatten()

                            r,p=pearsonr(results,testresponse)
                            r_squared=r*r
                            model='GB'
                            print(model,f'Fold:{rep}')
                            print(r*r)
                            Correlation.append({'time': timepoint, 'location':location, 'model':model, 'image': bands, 'r2':r_squared,'rep':rep})
                            rep=rep+1
                        

Scottsbluff
Crawfordsville
SatelliteImage_6bands_TP1_genotype.csv
TP1
RF Fold:1
0.008013990107678634
PSLR Fold:1
0.00011712129685528506
LR Fold:1
0.031106850405998285
LASSO Fold:1
nan
SVM Fold:1
0.023678487820748554
GB Fold:1
0.005776421131215432
RF Fold:2
0.023748171374826234
PSLR Fold:2
0.0013494461930006639
LR Fold:2
0.006419838547392266
LASSO Fold:2
nan
SVM Fold:2
0.028011420704772992
GB Fold:2
0.014433158990465045
RF Fold:3
0.00851771041194416
PSLR Fold:3
0.005527488089931371
LR Fold:3
0.0031242976084082815
LASSO Fold:3
nan
SVM Fold:3
0.028368767113143394
GB Fold:3
0.0013986783254159982
RF Fold:4
0.0005540764219373543
PSLR Fold:4
0.00044583239119065193
LR Fold:4
0.00884956704826768
LASSO Fold:4
nan
SVM Fold:4
0.00012686846103415468
GB Fold:4
0.002137728149386383
RF Fold:5
0.002845832691355161
PSLR Fold:5
0.032410623181584094
LR Fold:5
0.0015298723462831023
LASSO Fold:5
nan
SVM Fold:5
0.01576641797079882
GB Fold:5
0.0005788841502486524
SatelliteImage_6bands_TP2_genotype.csv
TP2
RF 

In [13]:
findcorr=pd.DataFrame(Correlation)
findcorr

,time,location,model,image,r2,rep
0,TP1,Crawfordsville,RF,Satellite Image,0.008014,1
1,TP1,Crawfordsville,PSLR,Satellite Image,0.000117,1
2,TP1,Crawfordsville,LR,Satellite Image,0.031107,1
3,TP1,Crawfordsville,LASSO,Satellite Image,NaN,1
4,TP1,Crawfordsville,SVM,Satellite Image,0.023678,1
...,...,...,...,...,...,...
805,TP1,Ames,PLSR,UAV,0.279773,5
806,TP1,Ames,LR,UAV,0.001330,5
807,TP1,Ames,LASSO,UAV,0.299321,5
808,TP1,Ames,SVM,UAV,0.194276,5


In [14]:
# findcorr=pd.DataFrame(Correlation)
findcorr.to_excel('Figure3/FinalCorrelation_totalstandcount_newversion_withReps.xlsx', index=False)

In [15]:
findcorr['image']=findcorr['image'].replace({'Satellite Image': 'Satellite'})
findcorr['location']=findcorr['location'].replace({'Movalley':'Missouri Valley','Crawfordsville':'Crawfordsville'})

In [29]:
findcorr

,time,location,model,image,r2,rep
0,TP1,Crawfordsville,RF,Satellite,0.008014,1
1,TP1,Crawfordsville,PSLR,Satellite,0.000117,1
2,TP1,Crawfordsville,LR,Satellite,0.031107,1
3,TP1,Crawfordsville,LASSO,Satellite,NaN,1
4,TP1,Crawfordsville,SVM,Satellite,0.023678,1
...,...,...,...,...,...,...
805,TP1,Ames,PLSR,UAV,0.279773,5
806,TP1,Ames,LR,UAV,0.001330,5
807,TP1,Ames,LASSO,UAV,0.299321,5
808,TP1,Ames,SVM,UAV,0.194276,5


In [30]:
dates=pd.read_excel('Figure1/Figure1A/DateofCollection.xlsx')

In [31]:
trial=pd.merge(findcorr, dates,left_on=['time','location', 'image'], right_on=['time','Location','Image'], how='outer')

In [32]:
trial['Date'] = pd.to_datetime(trial['Date'])

In [33]:
dates['Location'].value_counts()

Location
Scottsbluff        9
North Platte       9
Lincoln            9
Missouri Valley    9
Ames               9
Crawfordsville     9
Name: count, dtype: int64

In [34]:
trial['location'].value_counts()

location
Crawfordsville     270
Missouri Valley    270
Ames               270
Name: count, dtype: int64

In [36]:
trial.to_excel('Figure3/FinalCorrelationDates_totalstandcount.xlsx', index=False)

In [35]:
trial

,time,location,model,image,r2,rep,Location,Date,Image
0,TP1,Crawfordsville,RF,Satellite,0.008014,1.0,Crawfordsville,2022-07-10,Satellite
1,TP1,Crawfordsville,PSLR,Satellite,0.000117,1.0,Crawfordsville,2022-07-10,Satellite
2,TP1,Crawfordsville,LR,Satellite,0.031107,1.0,Crawfordsville,2022-07-10,Satellite
3,TP1,Crawfordsville,LASSO,Satellite,NaN,1.0,Crawfordsville,2022-07-10,Satellite
4,TP1,Crawfordsville,SVM,Satellite,0.023678,1.0,Crawfordsville,2022-07-10,Satellite
...,...,...,...,...,...,...,...,...,...
832,TP5,NaN,NaN,NaN,NaN,NaN,Lincoln,2022-09-19,Satellite
833,TP6,NaN,NaN,NaN,NaN,NaN,Lincoln,2022-09-27,Satellite
834,TP1,NaN,NaN,NaN,NaN,NaN,Lincoln,2022-07-13,UAV
835,TP2,NaN,NaN,NaN,NaN,NaN,Lincoln,2022-07-28,UAV


In [23]:
from dplython import X, group_by, summarize, DplyFrame
df_trial = DplyFrame(trial)
# grouped_trial = df_trial >> \
#   group_by(X.time, X.location, X.model, X.Date,X.image) >> \
#   summarize(r2_mean=X.r2.mean(), count=X.r2.count(), sd=X.r2.std(), r2_se=X.r2.std() / X.r2.count() ** 0.5)

In [24]:
# trial=pd.read_excel('Figure3/FinalCorrelationDates_totalstandcount.xlsx')

In [25]:
# from dplython import X, group_by, summarize, DplyFrame
# df_trial = DplyFrame(trial)
# grouped_trial = df_trial >> \
#   group_by(X.model,X.image) >> \
#   summarize(r2_mean=X.r2.mean(), count=X.r2.count(), sd=X.r2.std(), r2_se=X.r2.std() / X.r2.count() ** 0.5)

In [27]:
summary_trial = df_trial >> \
  group_by(X.model,X.image, X.Date, X.location) >> \
  summarize(r2_mean=X.r2.mean(), count=X.r2.count(), sd=X.r2.std(), r2_se=X.r2.std() / X.r2.count() ** 0.5)
            
            
summary_trial

,model,image,Date,location,r2_mean,count,sd,r2_se
0,GB,Satellite,2022-07-10,Crawfordsville,0.004865,5,0.005705,0.002552
1,GB,Satellite,2022-07-13,Missouri Valley,0.333849,5,0.143056,0.063977
2,GB,Satellite,2022-07-15,Ames,0.292052,5,0.082940,0.037092
3,GB,Satellite,2022-07-20,Crawfordsville,0.022732,5,0.040911,0.018296
4,GB,Satellite,2022-07-21,Missouri Valley,0.163757,5,0.140919,0.063021
...,...,...,...,...,...,...,...,...
157,SVM,UAV,2022-07-26,Crawfordsville,0.004929,5,0.005681,0.002541
158,SVM,UAV,2022-07-27,Missouri Valley,0.149781,5,0.163878,0.073289
159,SVM,UAV,2022-08-10,Ames,0.027472,5,0.030555,0.013665
160,SVM,UAV,2022-08-10,Missouri Valley,0.148434,5,0.136941,0.061242


In [28]:
summary_trial.to_excel('Figure3/FinalCorrelationDates_mean_totalstandcount.xlsx', index=False)

In [64]:
grouped_trial.to_csv('Figure3/modelperformance_totalstandcount.csv', index=False)

In [65]:
# Df=grouped_trial.pivot_table(index='model',
#               values='r2_mean', columns='image')

In [66]:
# Df['model']=Df.index

In [12]:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": "Nimbus Sans",
#     "font.size":12,
#     'axes.linewidth':2,
#     'xtick.major.size':5,
#     'ytick.major.size':5,
#     'ytick.major.width':2
# })


# ax=Df.plot(x='model', 
#         kind='bar', 
#         stacked=False, 
#          color={"Satellite": "lightgreen", "UAV": "darkgreen"}) 

# yticks=ax.get_yticks()
# xticks=ax.get_xticklabels()
# ax.set_yticklabels([round(x,2) for x in yticks])
# plt.ylabel(r'Average Prediction Accuracy ($R^2$)')
# plt.legend(title='Image type',frameon=False)
# ax.set_xticklabels(labels=xticks,rotation=0)
# # plt.savefig('AverageAcc_Models.png', dpi=350,bbox_inches='tight')

# plt.show()